### EIA API


In [1]:
# Energy Production and Usage
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### Crude Oil Production

In [2]:
# Crude Oil by State Group/Area
s_list = ['MCRFPCO2', 'MCRFPTX2', 'MCRFPND2', 'MCRFPNM2', 
          'MCRFPUS2', 'MCRFP3FM2']
s_ids = ';'.join([f'PET.{series}.M' for series in s_list])
url = (f'https://api.eia.gov/series/?api_key={eia_key}'+
       f'&series_id={s_ids}&start=198801')
r = requests.get(url).json()
df = pd.DataFrame({i['series_id']: {pd.to_datetime(f'{date}01'): 
                                    val for date, val in i['data']} 
                   for i in r['series']}).sort_index().astype('float')
df.to_csv(data_dir / 'oil_prod_raw.csv', index_label='date')

In [3]:
df = pd.read_csv(data_dir / 'oil_prod_raw.csv', index_col='date', 
                 parse_dates=True)
df['ND_NM_CO'] = df[['PET.MCRFPND2.M', 'PET.MCRFPNM2.M', 'PET.MCRFPCO2.M']].sum(axis=1)
df['USA'] = df['PET.MCRFPUS2.M']
df['TX'] = df['PET.MCRFPTX2.M']
df['GM'] = df['PET.MCRFP3FM2.M']
df['OTH'] = df['USA'] - df['TX'] - df['ND_NM_CO'] - df['GM']

data = df.loc['1989':,['ND_NM_CO', 'TX', 'OTH', 'GM', 'USA']].divide(1000, axis=1)
data.to_csv(data_dir / 'oil_prod.csv', index_label='date')
ltdate = dtxt(data.index[-1])['mon1']
ltval = data['USA'].iloc[-1]
text = (f'During {ltdate}, the US produced {ltval:.1f} million barrels per day.')
write_txt(text_dir / 'oil_prod.txt', text)
print(text)

During December 2021, the US produced 11.6 million barrels per day.


### Electricity Production by Source

In [4]:
# Electricity Production by Major Source
s_list = ['COW', 'NUC', 'TSN', 'HYC', 'HPS', 'NG', 'WND', 'GEO', 'WWW',
         'WAS', 'PEL', 'PC', 'ALL', 'AOR', 'OTH', 'SUN', 'OOG']
s_ids = ';'.join([f'ELEC.GEN.{series}-US-99.M' for series in s_list])
url = (f'https://api.eia.gov/series/?api_key={eia_key}'+
       f'&series_id={s_ids}')
r = requests.get(url).json()
df = pd.DataFrame({i['series_id']: {pd.to_datetime(f'{date}01'): 
                                    val for date, val in i['data']} 
                   for i in r['series']}).sort_index().astype('float')
df.to_csv(data_dir / 'elec_prod_raw.csv', index_label='date')

In [5]:
df = pd.read_csv(data_dir / 'elec_prod_raw.csv', index_col='date', 
                 parse_dates=True)

renew = ['SUN', 'HYC', 'WND', 'GEO', 'WAS', 'WWW']
renewables = [f'ELEC.GEN.{i}-US-99.M' for i in renew]
df['Renewables'] = df[renewables].sum(axis=1)

poth = ['PEL', 'PC', 'OTH', 'OOG', 'HPS']
petoth = [f'ELEC.GEN.{i}-US-99.M' for i in poth]
df['Petroleum and other'] = df[petoth].sum(axis=1)

bio = ['WWW', 'WAS']
biomass = [f'ELEC.GEN.{i}-US-99.M' for i in bio]
df['Biomass'] = df[biomass].sum(axis=1)

rename = {'ELEC.GEN.COW-US-99.M': 'Coal', 
          'ELEC.GEN.NUC-US-99.M': 'Nuclear', 
          'ELEC.GEN.NG-US-99.M': 'Natural Gas',
          'ELEC.GEN.HYC-US-99.M': 'Hydroelectric',
          'ELEC.GEN.SUN-US-99.M': 'Solar',
          'ELEC.GEN.WND-US-99.M': 'Wind',
          'ELEC.GEN.GEO-US-99.M': 'Geothermal'}
df.rename(rename, axis=1, inplace=True)

var = ['Coal', 'Natural Gas', 'Nuclear', 'Renewables', 'Petroleum and other',
       'Hydroelectric', 'Solar', 'Wind', 'Geothermal', 'Biomass']
data = df[var].rolling(12).sum().dropna().divide(1000, axis=1)
data.to_csv(data_dir / 'elec_prod.csv', index_label='date')

popgdp = pd.read_csv(data_dir / 'gdpjobslvl.csv', index_col='date', parse_dates=True)
pop = popgdp['pop'].dropna()
popch = (pop.iloc[-1] / pop.loc['2011'].mean() - 1) * 100
popinc = inc_dec_percent(popch)
gdp = popgdp['gdp'].dropna()
gdpch = (gdp.iloc[-1] / gdp.loc['2011'].mean() - 1) * 100
gdpinc = inc_dec_percent(gdpch)
major = ['Coal', 'Natural Gas', 'Nuclear', 'Renewables', 'Petroleum and other']
elec = data[major].sum(axis=1)
df2 = pd.DataFrame()
df2['elec'] = elec
df2['gdp'] = gdp
df2['pop'] = pop
df2['elecgdp'] = elec / gdp
df2['elecpop'] = elec / pop
df2 = ((df2 / df2.loc['2011-01-01']) - 1) * 100
df2.loc['2011':].to_csv(data_dir / 'elec.csv', index_label='date')

colors = {'gdp': 'green!60!blue', 
          'pop': 'blue!90!black', 
          'elec': 'red', 
          'elecpop': 'orange', 
          'elecgdp': 'cyan'}

adj = node_adjust(df2, colors)

#write_txt(text_dir / 'elec_nodes.txt', '')
node_file = open(text_dir / 'elec_nodes.txt', 'w')
for series, color in colors.items():
    if series in adj.keys():
        offset = adj[series]
    else:
        offset = 0
    node_file.write(end_node(df2[series].dropna(), color, percent=True))
node_file.close()

ltdate = dtxt(df.index[-1])['mon1']
ltval = data[major].iloc[-1].sum()
text = (f'Over the same period, the US population has {popinc} and real GDP has {gdpinc}. '+
        f'During the 12 months ending {ltdate}, the US generated {ltval:,.0f} billion kilowatt '+
        'hours of electricity.') 
write_txt(text_dir / 'elec_prod.txt', text)
print(text)

ngval = data['Natural Gas'].iloc[-1]
coalval = data['Coal'].iloc[-1]
nucval = data['Nuclear'].iloc[-1]
renval = data['Renewables'].iloc[-1]
text = (f'Over the year ending {ltdate}, {ngval:,.0f} billion kilowatt hours '+
        'of electricity were generated using natural gas (see\cbox{blue!40!cyan!60!white}), '+
        f'{coalval:,.0f} billion kilowatt hours were generated from coal '+
        f'(see\cbox{{brown}}), {nucval:,.0f} billion from nuclear (see\cbox{{red!80!blue!70!white}}), '+
        f'and {renval:,.0f} billion from renewable sources (see\cbox{{green!75!blue}}).')
write_txt(text_dir / 'elec_prod2.txt', text)
print(text)

hydval = data['Hydroelectric'].iloc[-1]
bioval = data['Biomass'].iloc[-1]
geoval = data['Geothermal'].iloc[-1]
sunval = data['Solar'].iloc[-1]
wndval = data['Wind'].iloc[-1]
text = (f'Among renewable energy sources, over the year ending {ltdate}, {hydval:,.0f} billion '+
        'kilowatt hours of electricity were generated with conventional hydroelectric (see\cbox{cyan}), '+
        f'{bioval:,.0f} billion kilowatt hours were generated from biomass (see\cbox{{brown!92!black}}), '+
        f'{geoval:,.0f} billion were generated from geothermal (see\cbox{{orange}}), '+
        f'{wndval:,.0f} billion from wind (see\cbox{{green!65!blue}}), '+
        f'and {sunval:,.0f} billion from solar (see\cbox{{yellow!70!orange}}).')
write_txt(text_dir / 'elec_prod3.txt', text)
print(text)

Over the same period, the US population has increased by 6.5 percent and real GDP has increased by 24.7 percent. During the 12 months ending December 2021, the US generated 4,116 billion kilowatt hours of electricity.
Over the year ending December 2021, 1,575 billion kilowatt hours of electricity were generated using natural gas (see\cbox{blue!40!cyan!60!white}), 899 billion kilowatt hours were generated from coal (see\cbox{brown}), 778 billion from nuclear (see\cbox{red!80!blue!70!white}), and 826 billion from renewable sources (see\cbox{green!75!blue}).
Among renewable energy sources, over the year ending December 2021, 260 billion kilowatt hours of electricity were generated with conventional hydroelectric (see\cbox{cyan}), 55 billion kilowatt hours were generated from biomass (see\cbox{brown!92!black}), 16 billion were generated from geothermal (see\cbox{orange}), 380 billion from wind (see\cbox{green!65!blue}), and 115 billion from solar (see\cbox{yellow!70!orange}).


### Electricity Retail Sales by Sector

In [6]:
# Electricity Retail Sales by Sector
s_list = ['RES', 'COM', 'IND']
s_ids = ';'.join([f'ELEC.SALES.US-{series}.M' for series in s_list])
url = (f'https://api.eia.gov/series/?api_key={eia_key}'+
       f'&series_id={s_ids}&start=200101')
r = requests.get(url).json()
df = pd.DataFrame({i['series_id']: {pd.to_datetime(f'{date}01'): 
                                    val for date, val in i['data']} 
                   for i in r['series']}).sort_index().astype('float')
df.to_csv(data_dir / 'elec_sales_raw.csv', index_label='date')

In [7]:
df = pd.read_csv(data_dir / 'elec_sales_raw.csv', index_col='date', 
                 parse_dates=True) / 1_000

rename = {'ELEC.SALES.US-RES.M': 'Residential', 
          'ELEC.SALES.US-COM.M': 'Commercial',
          'ELEC.SALES.US-IND.M': 'Industrial'}
df.rename(rename, axis=1, inplace=True)

data = df.rolling(12).sum().dropna()
data.to_csv(data_dir / 'elec_sales.csv', index_label='date')

cats = {'Commercial': 'blue!80!cyan', 'Industrial': 
        'black!35!white', 'Residential': 'green!90!black'}
nodes = '\n'.join([end_node(data[cat], col, digits='comma') 
                   for cat, col in cats.items()])
write_txt(text_dir / 'elec_ret_nodes.txt', nodes)

cl = {cat[:3]: c_line(col) for cat, col in cats.items()}
ltdt = dtxt(data.index[-1])['mon1']
lt = data.iloc[-1]
pr = data.loc['2019-12-01']
text = (f'Over the year ending {ltdt}, retail sales of electricity '+
        f'to the residential sector total {lt.Residential:,.0f} '+
        f'billion kilowatt hours, compared to {pr.Residential:,.0f} '+
        f'billion during 2019 {cl["Res"]}. Commercial sector '+
        f'electricity sales total {lt.Commercial:,.0f} billion kilowatt '+
        f'hours over the year ending {ltdt}, and {pr.Commercial:,.0f} '+
        f'billion kilowatt hours in 2019 {cl["Com"]}. Industrial '+
        f'sector sales total {lt.Industrial:,.0f} billion kilowatt '+
        f'hours in the latest 12 months of data and {pr.Industrial:,.0f} '+
        f'billion kilowatt hours in 2019 {cl["Ind"]}.')
write_txt(text_dir / 'elec_ret.txt', text)
print(text)

Over the year ending December 2021, retail sales of electricity to the residential sector total 1,477 billion kilowatt hours, compared to 1,440 billion during 2019 (see {\color{green!90!black}\textbf{---}}). Commercial sector electricity sales total 1,325 billion kilowatt hours over the year ending December 2021, and 1,361 billion kilowatt hours in 2019 (see {\color{blue!80!cyan}\textbf{---}}). Industrial sector sales total 987 billion kilowatt hours in the latest 12 months of data and 1,002 billion kilowatt hours in 2019 (see {\color{black!35!white}\textbf{---}}).
